In [1]:
import numpy as np 
import pandas as pd 
import json


In [2]:
rec = pd.read_csv('../../../data/model/recomendation.csv')

In [3]:
rec.head()

,client_code,product_code,score,rank
0,0,1,0.134833,1
1,0,2,0.060867,2
2,0,3,0.040529,3
3,0,4,0.038792,4
4,1,0,0.134833,1


In [5]:
grb_df = rec.loc[:, ['client_code','product_code','rank']].groupby('client_code').min().reset_index()
print(grb_df.nunique())


client_code     29501
product_code        5
rank                1
dtype: int64


In [6]:
grb_df.head()
print(grb_df.shape)

,client_code,product_code,rank
0,0,1,1
1,1,0,1
2,2,1,1
3,3,0,1
4,4,0,1


(29501, 3)


In [7]:
sub_df = pd.read_csv('../../../data/model/sub.csv')
sub_df.head()
print(sub_df.shape)

,policy_term,policy_payment_mode,policy_status,main_holder_smoker_flag,main_holder_gender,total_sum_assuared,premium_value,payment_method,client_code,product_code,month_rank,total_rider_prem,total_rider_sum_assuared,children_count,current_age,completed_duration,completed_ratio,is_cross_sell,preds
0,192,-0.020976,0.335611,0.018666,-0.011252,500000.0,36084.0,0.536784,24883,1,19,1080.0,120000,0,349,0,0.000000,0,1
1,180,-0.020976,-0.198882,0.018666,-0.011252,200000.0,24084.0,0.536784,21233,1,19,1524.0,550000,0,319,11,0.061111,1,1
2,240,-0.020976,-0.198882,0.018666,0.006204,700000.0,37884.0,0.536784,11408,1,19,0.0,0,0,305,30,0.125000,1,1
3,240,-0.020976,-0.198882,0.018666,0.006204,1570000.0,342972.0,0.536784,29166,0,19,0.0,0,0,407,10,0.041667,1,1
4,240,-0.020976,-0.198882,0.018666,-0.011252,528000.0,38100.0,0.536784,777,1,19,1848.0,1056000,0,613,31,0.129167,1,1


(318, 19)


In [8]:
fin_sub = sub_df.loc[:,['client_code','is_cross_sell']]


In [9]:
fin_sub['recomendation'] = fin_sub['client_code'].map(grb_df['product_code'])

In [10]:
fin_sub.head()
print(fin_sub.shape)

,client_code,is_cross_sell,recomendation
0,24883,0,0
1,21233,1,0
2,11408,1,1
3,29166,1,0
4,777,1,1


(318, 3)


In [11]:
product_dict = {'IN001': 0, 'HE001': 1, 'RE001': 2, 'ED001': 3, 'PR001': 4}

In [12]:
reverse_map = dict(zip(product_dict.values(), product_dict.keys()))
reverse_map

{0: 'IN001', 1: 'HE001', 2: 'RE001', 3: 'ED001', 4: 'PR001'}

In [13]:
with open('mappings.json','r') as pf:
    maps = json.load(pf)
client_rev_map = dict(zip(maps['client_code'].values(), maps['client_code'].keys()))

In [14]:
fin_sub['product_name'] = fin_sub['recomendation'].map(reverse_map)
fin_sub.head()

,client_code,is_cross_sell,recomendation,product_name
0,24883,0,0,IN001
1,21233,1,0,IN001
2,11408,1,1,HE001
3,29166,1,0,IN001
4,777,1,1,HE001


In [15]:
fin_sub['client_code'] = fin_sub['recomendation'].map(client_rev_map)

In [16]:
fin_sub.head()

,client_code,is_cross_sell,recomendation,product_name
0,C100000,0,0,IN001
1,C100000,1,0,IN001
2,C100001,1,1,HE001
3,C100000,1,0,IN001
4,C100001,1,1,HE001


In [17]:
fin_sub.drop('recomendation',axis=1,inplace=True)

In [18]:
fin_sub.rename(columns={'product_name': 'recomended_product'},inplace=True)

In [19]:
fin_df = fin_sub.groupby('client_code').last().reset_index()

In [20]:
fin_df.head()

,client_code,is_cross_sell,recomended_product
0,C100000,0,IN001
1,C100001,1,HE001
2,C100002,1,RE001


In [21]:
fin_df.to_csv('final_submission.csv', index=False)